<a href="https://colab.research.google.com/github/KNGLJordan/CDMO-project/blob/main/src/SAT/sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install z3
!pip install z3-solver

In [3]:
from z3 import *
from itertools import combinations

In [4]:
#the man the myth the legend: angelo quarta
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

In [12]:
# === PARAMETERS ===
#ask the user for n
n = int(input("Enter the number of teams: "))
weeks = n - 1
periods = n // 2
teams = list(range(n))  # 0-based indexing

solver = Optimize()

# === VARIABLES ===
# plays_home[t][w][p] = True if team t plays home in week w, period p
# plays_away[t][w][p] = True if team t plays away in week w, period p
plays_home = [[[Bool(f"home_team{t}_week{w}_period{p}") for p in range(periods)] for w in range(weeks)] for t in teams]
#if n = 6, therefore weeks = 5, periods = 3, then this will create a 3D list of size 6x5x3, of boolean variables
#for example: plays_home[0][1][2] = Bool("home_team0_week1_period2") = True if team 0 plays home in week 1, period 2
#in general: plays_away[t][w][p] = True if team t plays away in week w, period p
plays_away = [[[Bool(f"away_team{t}_week{w}_period{p}") for p in range(periods)] for w in range(weeks)] for t in teams]

# === 0. Each period (w, p) must have exactly one match: one home + one away
for w in range(weeks):
    for p in range(periods):
        home_teams = [plays_home[t][w][p] for t in teams] #in every iteration, home_teams will be a list of home teams that play in week w and period p
        away_teams = [plays_away[t][w][p] for t in teams] #same thing
        solver.add(exactly_one_he(home_teams, f"one_home_w{w}_p{p}")) #we want to have exactly one home team in week w and period p
        solver.add(exactly_one_he(away_teams, f"one_away_w{w}_p{p}"))
#there will be a total of (n−1)⋅(n//2)⋅2 = (n−1)⋅n, because for every period we impose two constraints (one for the away team and one for the home team)

# === 1. Every team plays once per week ===
for t in teams:
    for w in range(weeks):
        vars_in_week = [plays_home[t][w][p] for p in range(periods)] + [plays_away[t][w][p] for p in range(periods)]
        solver.add(exactly_one_np(vars_in_week, f"play_once_{t}_{w}"))
#there will be one constraint for each team and each week (n*(n-1) constraints)


# === 2. Every pair of teams plays exactly once ===
for i in teams:
    for j in teams:
        if i >= j: #otherwise we have all combinations, not just ordered ones
            continue
        match_conditions = [] #For each pair, we create an empty list match_conditions
        for w in range(weeks):
            for p in range(periods):
                # i home, j away OR j home, i away
                match_conditions.append(
                    Or(
                        And(plays_home[i][w][p], plays_away[j][w][p]),
                        And(plays_home[j][w][p], plays_away[i][w][p])
                    )
                )
        solver.add(exactly_one_he(match_conditions, f"one_match_{i}_{j}"))
        """ For every week w and period p, we define a condition representing "teams i and j play a match at this time" by adding to match_conditions:
                Either (i plays home AND j plays away)
                OR (j plays home AND i plays away)
                (not both at the same time, obviously)
            We set up a constraint that exactly one of these conditions must be true.
            This would mean that teams i and j play exactly once in the entire schedule."""
#there will be a total of (n 2) = (n(n-1) / 2) constraints (the number of combinations of n into groups of 2)


# === 3. A team plays at most twice in the same period (across weeks) ===
for team in teams:
    for period in range(periods):
        games_in_this_period = []

        for week in range(weeks):
            # Add 1 if team plays at home in this week/period
            games_in_this_period.append(plays_home[team][week][period])
            # Add 1 if team plays away in this week/period
            games_in_this_period.append(plays_away[team][week][period])

        # Ensure total is at most 2
        solver.add(at_most_k_seq(games_in_this_period, 2, 'at_most_2_games_period'))

#there will be a total of n*(n//2) constraints

# # === 4. Optional: Home/Away balance ===
# # Store home/away counts for each team (needed later for printing)
# home_counts = []  # List to store home count expressions
# away_counts = []  # List to store away count expressions
# imbalances = []   # List to store imbalance variables

# for t in teams:
#     # Count home games for team t
#     home_count = Sum([plays_home[t][w][p] for w in range(weeks) for p in range(periods)])
#     # Count away games for team t
#     away_count = Sum([plays_away[t][w][p] for w in range(weeks) for p in range(periods)])

#     # Store these for later use in printing
#     home_counts.append(home_count)
#     away_counts.append(away_count)

#     # Create imbalance variable for team t
#     imbalance = Int(f"imbalance_{t}")
#     imbalances.append(imbalance)

#     # Add constraint: imbalance = |home_count - away_count|
#     solver.add(imbalance == If(home_count >= away_count,
#                               home_count - away_count,
#                               away_count - home_count))

# # Minimize the maximum imbalance across all teams
# # === Compute total imbalance score ===
# total_imbalance = Int("total_imbalance")
# solver.add(total_imbalance == Sum(imbalances) - n)

#there will be n+1 constraints for this optimization

# === Minimize total imbalance score ===
# solver.minimize(total_imbalance)


# === SOLVE ===
if solver.check() == sat:
    model = solver.model()

    # Create a 2D table to store matches
    schedule = [[None for _ in range(weeks)] for _ in range(periods)]

    # Fill in the schedule table
    for w in range(weeks):
        for p in range(periods):
            for i in teams:
                if model.evaluate(plays_home[i][w][p], model_completion=True):
                    for j in teams:
                        if model.evaluate(plays_away[j][w][p], model_completion=True):
                            # Store as "home v away"
                            schedule[p][w] = f"{i+1} v {j+1}"

    # Print table header
    print(" " * 14, end="")
    for w in range(weeks):
        print(f"Week {w+1}".center(14), end="")
    print()

    # Print separator line
    print(" " * 14 + "-" * (14 * weeks))

    # Print each period row
    for p in range(periods):
        print(f"Period {p+1}:".ljust(10) + " |", end="")
        for w in range(weeks):
            print(f"    {schedule[p][w]}    |", end="")
        print()

        # Print separator line
        print(" " * 10 + " " + "-" * (14 * weeks))

    # Print imbalance information
    # print("\nImbalances:")
    # for t in teams:
    #     h = model.evaluate(home_counts[t])
    #     a = model.evaluate(away_counts[t])
    #     d = model.evaluate(imbalances[t])
    #     print(f"Team {t+1}: Home={h}, Away={a}, Δ={d}")
    # print("\nTotal Balance Score:", model.evaluate(total_imbalance))
else:
    print("Unsat.")

Enter the number of teams: 6
                  Week 1        Week 2        Week 3        Week 4        Week 5    
              ----------------------------------------------------------------------
Period 1:  |    None    |    None    |    None    |    5 v 6    |    None    |
           ----------------------------------------------------------------------
Period 2:  |    None    |    None    |    None    |    None    |    None    |
           ----------------------------------------------------------------------
Period 3:  |    None    |    None    |    None    |    6 v 6    |    None    |
           ----------------------------------------------------------------------
